In [ ]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import json
import math
import re
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict
from datasketch import MinHash,MinHashLSH
from datasketch import MinHashLSHForest,MinHashLSHEnsemble
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter,defaultdict
from sklearn.utils import resample
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import resample
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram


In [ ]:
file_path = 'TVs-all-merged.json' 
import json
with open(file_path, 'r') as file:
    data = json.load(file)
    
# Extract the titles and model IDs for all products in the dataset
product_titles = []
model_ids = []
for products in data.values():
    for product in products:
        product_titles.append(product['title'])
        model_ids.append(product['modelID'])

# Map the modelID to title (considering multiple instances)
original_title_to_modelid = dict(zip(product_titles, model_ids))

# Use indices as keys to maintain uniqueness
index_to_modelid = {idx: model_id for idx, model_id in enumerate(model_ids)}
    
print(len(model_ids))

In [ ]:
# normalize the terms mentioned by MSMP+ paper 
# all spaces and non-alphanumeric tokens in front of the units are removed
# all upper case is transformed into lower case
def normalize_terms(text):
    # Define patterns for 'inch' and 'hz' with possible preceding non-alphanumeric characters or spaces
    inch_patterns = [r'\s*[\W_]*inch', r'\s*[\W_]*inches', r'\s*[\W_]*”', r'\s*[\W_]*-inch', r'\s*[\W_]* inch']
    hz_patterns = [r'\s*[\W_]*Hertz', r'\s*[\W_]*hertz', r'\s*[\W_]*Hz', r'\s*[\W_]*HZ', r'\s*[\W_]* hz', r'\s*[\W_]*-hz', r'\s*[\W_]*hz']
    
    # Replace with normalized terms
    for pattern in inch_patterns:
        text = re.sub(pattern, '', text)
    for pattern in hz_patterns:
        text = re.sub(pattern, 'hz', text)
        
    return text

def extract_features(features_map):
    desired_features = ['Refresh Rate', 'Brand', "Recommended Resolution", "Screen Size", "Screen Size Class", "Screen Refresh Rate", "Vertical Resolution", "Maximum Resolution", "Display Size"]

    # Convert desired features to lowercase for case-insensitive comparison
    desired_features_lower = [feature.lower() for feature in desired_features]

    # Convert the keys of features_map to lowercase
    features_map_lower = {key.lower(): value for key, value in features_map.items()}

    # Extract only the desired features, removing "hz" if the value ends with it
    features_str = ' '.join([
        features_map_lower.get(feature, '').rstrip('hz').rstrip() if feature in features_map_lower else ''
        for feature in desired_features_lower
    ])

    # features_str = features_map
    return features_str


stop_words = set(["yes", "no", "class", "best", "buy", "we", "it", "refurbished", "to", "a", "neweggcom", "tv", "smart", "series", "model", "tvs", "with", "apps", "x"])  # Add more words as needed

def preprocess_title(title, features_map):

    # title = title + ' ' + extract_features(features_map)
    # for x in range(10):
    #     title = title + ' ' + (model_id+ str(x))
    # Preprocess the title
    title = title.lower()
    title = re.sub(r'[^\w\s]', '', title)
    title = title.replace('hdtv', 'hd')
    title = title.replace('ledlcd', 'led lcd')

    # Normalize specific terms
    title = normalize_terms(title)

    combined = title
    words = combined.split()

    # Remove stop words
    filtered_words = [word for word in words if word not in stop_words]

    # duplicate first 3 words and last 1 word 
    if len(filtered_words) >= 4:
        new_words = []
        for x in range(3):
            for y in range(3):
                new_words.append(filtered_words[x]+str(y))
        for x in range(8):
            new_words.append(filtered_words[-1]+str(x))
        filtered_words = new_words + filtered_words

    # Join the words back into a string
    return ' '.join(filtered_words)

# Apply preprocessing to all product titles and include features
preprocessed_titles = [preprocess_title(product['title'], product['featuresMap']) for products in data.values() for product in products]

#print(preprocessed_titles)
#print(len(preprocessed_titles))

#for i in range(301,310):
#    print(preprocessed_titles[i])  

In [ ]:
# create sets of k-shingles for the title 
def create_shingles(text, k=6):
    """Create a set of k-shingles from the given text."""
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i+k]
        shingles.add(shingle)
    return shingles

# Applying shingling to each preprocessed title
shingles_per_title = [create_shingles(title) for title in preprocessed_titles]

# check the shingles for the first title
for i in range(1):  # Change the range as needed
    print(f"Title {i+1}: {preprocessed_titles[i]}")
    print(f"Shingles: {shingles_per_title[i]}\n")
    
#define a hash function that convert each k-shingle for each titile into a bucket number takes only 4 bytes 
def simple_hash(shingle):
    """A simple hash function to convert a shingle to a bucket number."""
    hash_value = 0
    for char in shingle:
        hash_value = hash_value * 31 + ord(char)
    return hash_value

# Hashing the shingles for each title
hashed_shingles_per_title = []
for shingles in shingles_per_title:
    hashed_shingles = set()
    for shingle in shingles:
        hashed_shingle = simple_hash(shingle)
        hashed_shingles.add(hashed_shingle)
    hashed_shingles_per_title.append(hashed_shingles)

# Check the hashed shingles for the first title
#for i in range(1):  # Change the range as needed
#    print(f"Title {i+1}: {preprocessed_titles[i]}")
#    print(f"Hashed Shingles: {hashed_shingles_per_title[i]}\n")



In [ ]:
## create the characteristic matrix from the shingles 
# Identify all unique hashed shingles across all titles
all_unique_shingles = set.union(*hashed_shingles_per_title)

# Create an empty characteristic matrix
char_matrix = np.zeros((len(all_unique_shingles), len(hashed_shingles_per_title)), dtype=int)

# Create a mapping of shingle to row index
shingle_to_index = {shingle: idx for idx, shingle in enumerate(all_unique_shingles)}

# Fill the matrix
for title_idx, shingles in enumerate(hashed_shingles_per_title):
    for shingle in shingles:
        row_idx = shingle_to_index[shingle]
        char_matrix[row_idx][title_idx] = 1

# Check the first few columns and rows of the matrix
print("First few columns and rows of the characteristic matrix:")
print(char_matrix[:5, :5])  # Adjust the range as needed for rows and columns
# Check if the matrix contains any 1s
contains_one = np.any(char_matrix == 1)



In [ ]:
#now create an ALTERNATIVE WAY of minhashing 
#using random hash function to simulate the effect of random permutation 
def get_hash_functions(n, max_row):
    """Generate n random hash functions."""
    def hash_factory(a, b, p=2**33-355, m=max_row):
        return lambda x: (a * x + b) % p % m

    np.random.seed(42)  # For reproducibility
    return [hash_factory(np.random.randint(1, max_row), np.random.randint(0, max_row)) for _ in range(n)]

n = 500  # Number of hash functions
num_titles = len(hashed_shingles_per_title)
max_row = len(all_unique_shingles)

# Generate n hash functions
hash_functions = get_hash_functions(n, max_row)

# Initialize the signature matrix with a very large integer (as infinity)
large_integer = 2**32 - 1
signature_matrix = np.full((n, num_titles), large_integer, dtype=int)

# Fill the signature matrix
for r, shingle in enumerate(all_unique_shingles):
    hash_values = [h(r) for h in hash_functions]
    for c in range(num_titles):
        if char_matrix[shingle_to_index[shingle]][c] == 1:
            for i in range(n):
                signature_matrix[i][c] = min(signature_matrix[i][c], hash_values[i])

# Check the first few rows and columns of the signature matrix
print("First few rows and columns of the signature matrix:")
print(signature_matrix[:5, :5])  # Adjust the range as needed for rows and columns


In [ ]:
# construct candidate pairs by applying the LSH technique 
def hash_vector_to_bucket(vector, num_buckets=2**24): #A larger range can help reduce likelihood of hash collisions
    """Hash a vector to a bucket number."""
    hash_value = hash(tuple(vector))
    return hash_value % num_buckets

def apply_lsh(signature_matrix, b, r, num_buckets=2**24):
    """Apply LSH to find candidate pairs."""
    n, num_titles = signature_matrix.shape
    assert n == b * r, "b * r must equal n"

    candidate_pairs = set()
    for band in range(b):
        buckets = defaultdict(list)
        start_row = band * r
        end_row = start_row + r

        # Hash each column vector within this band
        for c in range(num_titles):
            column_slice = signature_matrix[start_row:end_row, c]
            bucket = hash_vector_to_bucket(column_slice, num_buckets)
            buckets[bucket].append(c)

        # Add pairs from the same bucket to candidate pairs
        for bucket_items in buckets.values():
            if len(bucket_items) > 1:
                for pair in itertools.combinations(bucket_items, 2):
                    candidate_pairs.add(pair)

    return candidate_pairs

# Choose b and r
b = 50 # Number of bands
r = 10 # Number of rows per band
assert b * r == n, "b * r must equal n"

# Calculate threshold t
t = (1/b)**(1/r)
print(f"Threshold t: {t}")

# Apply LSH
candidate_pairs = apply_lsh(signature_matrix, b, r)

# Count the total number of candidate pairs
total_candidate_pairs = len(candidate_pairs)

# Print the result
print(f"Total number of candidate pairs: {total_candidate_pairs}")


# Print some of the candidate pairs
print("Some candidate pairs:")
for i, pair in enumerate(list(candidate_pairs)[:10]):  # Display first 10 candidate pairs
    print(f"Pair {i+1}: {pair}")

In [ ]:
##apply clustering on the candidate duplicate pairs 

# Count the number of products for each model ID
model_id_counts = {model_id: len(products) for model_id, products in data.items()}
# Calculate the total number of actual duplicate pairs
total_actual_duplicates = sum(count * (count - 1) // 2 for count in model_id_counts.values() if count > 1)
print(f"Total actual duplicates: {total_actual_duplicates}")

def is_duplicate(pair_indices, index_to_modelid):
    model_id1 = index_to_modelid[pair_indices[0]]
    model_id2 = index_to_modelid[pair_indices[1]]
    return model_id1 == model_id2


In [ ]:
pair_indice = list(candidate_pairs)[14]
print(pair_indice)
dup = is_duplicate(pair_indice, index_to_modelid)
print(dup)
print(index_to_modelid[pair_indice[0]])
print(index_to_modelid[pair_indice[1]])
print(preprocessed_titles[pair_indice[0]])
print(preprocessed_titles[pair_indice[1]])

# minhash_obj = minhashes[pair_indice[0]]

# # To view the hash values
# print(minhash_obj.hashvalues)


In [ ]:
candidate_titles = []
candidate_indices = []
for i in candidate_pairs:
    if i[0] not in candidate_indices:
        candidate_indices.append(i[0])
        candidate_titles.append(preprocessed_titles[i[0]])
    if i[1] not in candidate_indices:
        candidate_indices.append(i[1])
        candidate_titles.append(preprocessed_titles[i[1]])

print(len(candidate_titles))

In [ ]:
vectorizer = TfidfVectorizer()
# Create a TfidfVectorizer with q-gram (bigram) configuration
# vectorizer = TfidfVectorizer(ngram_range=(2, 2))  # This will create bigrams
tfidf_matrix = vectorizer.fit_transform(candidate_titles)
print(tfidf_matrix.shape)

similarity_matrix = cosine_similarity(tfidf_matrix)

In [ ]:

def clustering(similarity_matrix, candidate_indices):

    # Uses the average of the distances of each observation of the two sets.
    Z = linkage(similarity_matrix, method='average', metric='cosine')

    max_dist = 0.6 # can adjust this threshold 
    clusters = fcluster(Z, max_dist, criterion='inconsistent')

    cluster_dict = {}
    for idx, cluster_id in enumerate(clusters):
        if cluster_id not in cluster_dict:
            cluster_dict[cluster_id] = []
        cluster_dict[cluster_id].append(candidate_indices[idx])

    # Filter clusters with more than one title (potential duplicates)
    duplicate_clusters = {k: v for k, v in cluster_dict.items() if len(v) > 1}
    return duplicate_clusters


duplicate_clusters = clustering(similarity_matrix, candidate_indices)
print(f"Number of duplicate clusters: {len(duplicate_clusters)}")
#print(duplicate_clusters)


def all_true_duplicates(candidate_indices, index_to_modelid):
    true_duplicates = []
    for pair in itertools.combinations(candidate_indices, 2):
        if is_duplicate(pair, index_to_modelid):
            true_duplicates.append(pair)
    return true_duplicates


false_negatives = 0
for model_id, products in data.items():
    if len(products) > 1:
        for pair in itertools.combinations(products, 2):
            pair_indices = (product_titles.index(pair[0]['title']), product_titles.index(pair[1]['title']))
            if is_duplicate(pair_indices, index_to_modelid):
                if not any(pair_indices[0] in cluster and pair_indices[1] in cluster for cluster in duplicate_clusters.values()):
                    false_negatives += 1

true_positives, false_positives = 0,0

for cluster in duplicate_clusters.values():
    for pair in itertools.combinations(cluster, 2):
        if is_duplicate(pair, index_to_modelid):
            true_positives += 1
        else:
            false_positives += 1
            

        
PC = true_positives / total_actual_duplicates
PQ = true_positives / total_candidate_pairs
f1star = 2* PC * PQ / (PC+PQ)
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1 = 2 * precision * recall / (precision + recall)

print(f"FN:{false_negatives:.3f}")
print(f"TP:{true_positives:.3f}")  
print(f"Candidate Pairs:{total_candidate_pairs}")
print(f"Pair completeness:{PC:.3f}")
print(f"Pair Quality:{PQ:.3f}")
print(f"F1star:{f1star:.3f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")


In [ ]:
vectorizer = TfidfVectorizer()
# Create a TfidfVectorizer with q-gram (bigram) configuration
tfidf_matrix_full = vectorizer.fit_transform(preprocessed_titles)
print(tfidf_matrix_full.shape)

def estimate_jaccard_similarity(pair_indices, minhashes, num_perm):
    minhash1 = minhashes[pair_indices[0]]
    minhash2 = minhashes[pair_indices[1]]
    count = sum(1 for i in range(num_perm) if minhash1.hashvalues[i] == minhash2.hashvalues[i]) / num_perm
    return count

# Number of bootstrap iterations
n_bootstrap = 5 

# Bootstrapping and evaluation
precision_scores, recall_scores, f1_scores = [], [], []

for i in range(n_bootstrap):
   true_positives, false_positives = 0, 0
   for pair_indices in candidate_pairs:
      #  if estimate_jaccard_similarity(pair_indices, minhashes, num_perm) > 0.6:
         if cosine_similarity(tfidf_matrix_full[pair_indices[0]], tfidf_matrix_full[pair_indices[1]]) > 0.65:

            if is_duplicate(pair_indices, index_to_modelid):
               true_positives += 1
            else:
               false_positives += 1

    # Calculate precision, recall, and F1 score
   precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
   recall = true_positives / total_actual_duplicates if total_actual_duplicates > 0 else 0
   f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Store the scores
   precision_scores.append(precision)
   recall_scores.append(recall)
   f1_scores.append(f1)

    # Calculate average metrics over all bootstraps
   avg_precision = sum(precision_scores) / n_bootstrap
   avg_recall = sum(recall_scores) / n_bootstrap
   avg_f1 = sum(f1_scores) / n_bootstrap

   avg_precision, avg_recall, avg_f1
    
print(f"FN:{false_negatives:.3f}")
print(f"TP:{true_positives:.3f}")  
print(f"Candidate Pairs:{total_candidate_pairs}")
print(f"Pair completeness:{PC:.3f}")
print(f"Pair Quality:{PQ:.3f}")
print(f"F1star:{f1star:.3f}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")
